In [1]:
!pip install -U "bitsandbytes>=0.46.1"

import torch
import bitsandbytes
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import prepare_model_for_kbit_training, LoraConfig, TaskType, get_peft_model


MAX_LENGTH = 128
MODEL = 'mistralai/Mistral-7B-v0.1'

In [2]:
dataset = load_dataset('go_emotions')
labels = dataset['train'].features['labels'].feature.names
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
labels

['admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [4]:
dataset['train'][0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [27],
 'id': 'eebbqej'}

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.pad_token = tokenizer.eos_token

def preprocess(example):
    tokenized = tokenizer(example['text'], padding=False, truncation=True, max_length=MAX_LENGTH)

    example_labels = [0.0] * len(labels)
    for label_id in example['labels']:
        example_labels[label_id] = 1.0

    tokenized['labels'] = example_labels
    return tokenized

dataset_encoded = dataset.map(preprocess, remove_columns=dataset['train'].column_names, load_from_cache_file=False)
dataset_encoded.set_format('torch')


Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

In [6]:
train_dataset = dataset_encoded['train']
eval_dataset = dataset_encoded['validation']
test_dataset = dataset_encoded['test']

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL,
    num_labels=len(labels),
    label2id=label2id,
    id2label=id2label,
    quantization_config=bnb_config,
    device_map='auto',
    problem_type='multi_label_classification'
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

peft_confing = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    target_modules=['q_proj', 'v_proj']
)

model = get_peft_model(model, peft_confing)

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

MistralForSequenceClassification LOAD REPORT from: mistralai/Mistral-7B-v0.1
Key            | Status     | 
---------------+------------+-
lm_head.weight | UNEXPECTED | 
score.weight   | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [13]:
from sklearn.metrics import f1_score, roc_auc_score

# BATCH_SIZE = 8
EPOCHES = 1
OUTPUT_DIR = '/content/drive/MyDrive/data/train'
LEARNING_RATE = 2e-4

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    scores = torch.sigmoid(torch.tensor(logits)).numpy()
    predictions = (scores > 0.5).astype(int)
    labels = labels.astype(int)

    f1_micro = f1_score(labels, predictions, average='micro')
    f1_macro = f1_score(labels, predictions, average='macro')

    try:
        roc_score = roc_auc_score(labels, scores, average='macro')
    except ValueError:
        roc_score = 0.0

    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'roc_auc': roc_score,
    }

class MultiLabelDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        raw_labels = [f.pop('labels') for f in features]
        batch = super().__call__(features)

        if isinstance(raw_labels[0], torch.Tensor):
            batch['labels'] = torch.stack(raw_labels).to(dtype=torch.float32)
        else:
            batch['labels'] = torch.tensor(raw_labels, dtype=torch.float32)

        return batch


BATCH_SIZE = 16

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHES,
    weight_decay=0.01,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1_micro',
    logging_steps=50,
    bf16=True,
    # fp16=False,
    gradient_checkpointing=True,
    gradient_accumulation_steps=4,
    report_to='none',
    optim='paged_adamw_8bit',
    dataloader_num_workers=2,
    dataloader_pin_memory=True,
)

data_collator = MultiLabelDataCollator(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Starting Training...")
trainer.train()

Starting Training...


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Roc Auc
1,0.335314,0.080359,0.575605,0.415027,0.937720


TrainOutput(global_step=679, training_loss=0.391287864044594, metrics={'train_runtime': 6884.9505, 'train_samples_per_second': 6.305, 'train_steps_per_second': 0.099, 'total_flos': 6.419547760430285e+16, 'train_loss': 0.391287864044594, 'epoch': 1.0})